In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# !rm -fr './kaggle/working/train'
# !rm -fr './model_chain_rn101_10.pkl'

In [ ]:
import fastai
import pandas as pd
import torch
import os
import dill
from fastai.imports import *
from fastai.vision import *
from fastai.vision.all import *
from torchvision import datasets, transforms
from fastai.metrics import error_rate

In [ ]:
# torch.cuda.get_device_name(0)

## Folder Paths

In [ ]:
## Defining Directories
path = Path('../input/hotel-id-2021-fgvc8')

sample_sub = path/'sample_submission.csv'
labels = path/'train.csv'

## Reading csv submission file

In [ ]:
train_data = pd.read_csv(labels).sample(frac=1, random_state = 2021).reset_index(drop = True)
train_data.head(5)

In [ ]:
# train_dir = './kaggle/working/train/'
train_dir = '../input/fgvc8hoteltrain512/'
train_data['image_path'] = train_dir + train_data.chain.astype('str') + '/' + train_data.image
train_data[:5]

## Unzip resized Images

In [ ]:
# !tar xzvf /kaggle/input/hotelid-224/kaggle_hotelid_train_imgs_224x224.tgz

## Creating dataset per chain

In [ ]:
batch_size = 32
def create_dataLoader(train_subset):
    dataset = ImageDataLoaders.from_df(df = train_subset[['image_path', 'hotel_id']],
                                   path = '.',
                                   folder = '.',
                                   valid_pct=0.2,
                                   item_tfms=Resize(224, method='pad', pad_mode='reflection'),
                                   batch_tfms=aug_transforms(size=224),
                                   bs=batch_size)
    return dataset

In [ ]:
category_dir,folder_label =[], []
folder_list = os.listdir(train_dir)
len(folder_list)

In [ ]:
# for i in range(len(folder_list)):
#     img_dir = str(train_dir) + str(folder_list[i] + '/')
#     if  os.listdir(img_dir) :
#         # skip directories
#         continue
#     else:
#         print(img_dir)
#         os.rmdir(img_dir)
        

In [ ]:
for i in range(len(folder_list)):
    img_dir = str(train_dir) + str(folder_list[i] + '/')
    if  os.listdir(img_dir) :
        print(folder_list[i])
        df_subset = train_data.copy()
        is_chain =  df_subset.chain==i
        train_subset = df_subset[is_chain]
        dataset_chain = create_dataLoader(train_subset)
        learn = cnn_learner(dataset_chain, resnet101, metrics=[accuracy, error_rate],   opt_func=QHAdam).to_fp16()
        %time
        learn.fine_tune(12, 0.005, freeze_epochs=3)
        model_name ='model_chain_rn101'
        learn.export(f'{model_name}_{folder_list[i]}.pkl', pickle_module=dill)
        

In [ ]:
learn.export(f'{model_name}_{i}.pkl', pickle_module=dill)